In [48]:
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor
import numpy as np

In [49]:
import math
import sys
sys.path.append("../CSP/")
from sudoku import Sudoku


# sudoku = Sudoku.make_sudoku()
sudoku = Sudoku(init=[[4, 0, 0, 0], [0, 0, 0, 0], [1, 0, 4, 0], [0, 0, 0, 2]])

In [50]:
# sudoku_size = 4
# print(np.base_repr(15, 4))
# print(type(np.base_repr(15, 4)))
# print(sum([int(char) for char in np.base_repr(15, 4)]))
# for i in range(sudoku_size ** sudoku_size):
#     print(True if sum([int(char) for char in np.base_repr(i, sudoku_size)]) == 6 else False, i)


In [51]:
sudoku_size = 4
root_sudoku_size = int(math.sqrt(sudoku_size))
pow_two_mask_except_zero = [] #True if . False in Otherwise
for i in range(sudoku_size ** sudoku_size):
    base_sudoku_size_state = [int(char) for char in np.base_repr(i, sudoku_size)]
    pow_two_mask_except_zero.append(True if sum(base_sudoku_size_state) == 6 and len([num for num in base_sudoku_size_state if num != 0]) == 3 else False)

enforce_rate = 0.99
factor_prob = np.where(pow_two_mask_except_zero, enforce_rate, 1 - enforce_rate)
number_of_random_variable_state = [sudoku_size for _ in range(sudoku_size)]

In [52]:
graph = FactorGraph()
graph.add_nodes_from(["{}_{}".format(y, x) for y in range(sudoku_size) for x in range(sudoku_size)])
# print("="*50)
#column constraint
for y in range(sudoku_size):
    target_node = ["{}_{}".format(y, x) for x in range(sudoku_size)]
    # print(target_node)
    factor = DiscreteFactor(target_node, number_of_random_variable_state, factor_prob)
    graph.add_factors(factor)
    graph.add_edges_from([(target_node[i], factor) for i in range(sudoku_size)])
# print("="*50)
# row constraint
for x in range(sudoku_size):
    target_node = ["{}_{}".format(y, x) for y in range(sudoku_size)]
    # print(target_node)
    factor = DiscreteFactor(target_node, number_of_random_variable_state, factor_prob)
    graph.add_factors(factor)
    graph.add_edges_from([(target_node[i], factor) for i in range(sudoku_size)])
# print("="*50)
#block constraint
for block_y in range(root_sudoku_size):
    for block_x in range(root_sudoku_size):
        target_node = ["{}_{}".format(y, x) for y in range(block_y * root_sudoku_size, (block_y + 1) * root_sudoku_size) for x in range(block_x * root_sudoku_size, (block_x + 1) * root_sudoku_size)]
        # print(target_node)
        factor = DiscreteFactor(target_node, number_of_random_variable_state, factor_prob)
        graph.add_factors(factor)
        graph.add_edges_from([(target_node[i], factor) for i in range(sudoku_size)])
graph.check_model()


True

In [53]:
from pgmpy.inference import BeliefPropagation
bp = BeliefPropagation(graph)
bp.calibrate()
evidence = {}
variable = []
for y in range(sudoku_size):
    for x in range(sudoku_size):
        num = sudoku.init[y][x]
        if num != 0:
            evidence["{}_{}".format(y, x)] = num
        else:
            variable.append("{}_{}".format(y, x))
bp.map_query(variable, evidence)

In [56]:
for y in range(sudoku_size):
    for x in range(sudoku_size):
        if sudoku.init[y][x] == 0:
            target_node = "{}_{}".format(y, x)
            factor = DiscreteFactor([target_node], [sudoku_size], np.ones(sudoku_size) / sudoku_size)
            graph.add_factors(factor)
            graph.add_edges_from([(target_node, factor)])
        else:
            target_node = "{}_{}".format(y, x)
            prob = np.zeros(sudoku_size)
            prob[sudoku.init[y][x] - 1] += 1.0
            factor = DiscreteFactor([target_node], [sudoku_size], prob)
            graph.add_factors(factor)
            graph.add_edges_from([(target_node, factor)])
graph.check_model()

True

In [57]:
from pgmpy.inference import BeliefPropagation
bp = BeliefPropagation(graph)
bp.calibrate()

/home/takeshi/.pyenv/versions/3.9.4/envs/snn_bayes/lib/python3.9/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:796: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi1.values


In [58]:
bp.query(["{}_{}".format(y, x) for y in range(sudoku_size) for x in range(sudoku_size)])

/home/takeshi/.pyenv/versions/3.9.4/envs/snn_bayes/lib/python3.9/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:796: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi1.values


In [11]:
mm = graph.to_markov_model()

In [1]:
from pgmpy.readwrite import UAIWriter
writer = UAIWriter(mm)
writer.write_uai('models/4_4_sudoku_[[4, 0, 0, 0], [0, 0, 0, 0], [1, 0, 4, 0], [0, 0, 0, 2]]_markov.xml')

/home/takeshi/.pyenv/versions/3.9.4/envs/snn_bayes/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'mm' is not defined

In [1]:
from pgmpy.readwrite import UAIReader
reader = UAIReader('models/4_4_sudoku_[[4, 0, 0, 0], [0, 0, 0, 0], [1, 0, 4, 0], [0, 0, 0, 2]]_markov.xml')
mm = reader.get_model()

/home/takeshi/.pyenv/versions/3.9.4/envs/snn_bayes/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from pgmpy.inference import BeliefPropagation
infer = BeliefPropagation(mm)

MemoryError: Unable to allocate 1.00 PiB for an array with shape (140737488355328,) and data type float64

In [2]:
from pgmpy.sampling import GibbsSampling
gibbs_chain = GibbsSampling(mm)

104220955it [1:45:35, 16451.23it/s]/s]
  0%|          | 0/64 [1:45:35<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
sample = gibbs_chain.sample(size=100, return_tupe='dataframe')